In [1]:
%pwd

'c:\\Users\\odz-2\\Desktop\\itü\\bitirme\\2nd part\\exercise-chatbot-project\\exercise-chatbot-project\\notebooks'

In [2]:
import os
os.chdir("../")

In [3]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv

# Load API keys
load_dotenv()

True

In [4]:
GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [5]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

c:\Users\odz-2\anaconda3\envs\excbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "fitness-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # Dimension for 'all-MiniLM-L6-v2'
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)


In [9]:
from langchain_community.document_loaders import PyPDFLoader

loaders = {
    ".txt": TextLoader,
    ".pdf": PyPDFLoader
}


def create_loader(file_path):
    ext = os.path.splitext(file_path)[1]
    if ext in loaders:
        return loaders[ext](file_path)
    return None

path = "data/knowledge_base/"
docs = []

for file in os.listdir(path):
    loader = create_loader(os.path.join(path, file))
    if loader:
        docs.extend(loader.load())

print(f"Loaded {len(docs)} documents (7 TXTs and 2 PDFs).")


Loaded 166 documents (7 TXTs and 2 PDFs).


In [10]:
def text_split(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=700,
        chunk_overlap=100
        )
    text_chunks = text_splitter.split_documents(docs)
    return text_chunks\
    


In [11]:
texts_chunks = text_split(docs)
print(f"Number of chunks: {len(texts_chunks)}")

Number of chunks: 549


In [13]:
def download_embeddings():
    model_name = "all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        
    )
    return embeddings

embedding = download_embeddings()

In [ ]:
#converting the text chunks into embeddings and storing them in Pinecone vector store

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunks,
    embedding=embedding,
    index_name=index_name
)

In [15]:
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embedding,
    index_name=index_name
)

In [ ]:
def test_search(query):
    # k=3 means "find the top 3 most relevant paragraphs"
    results = docsearch.similarity_search(query, k=3)
    for i, res in enumerate(results):
        print(f"\n--- Result {i+1} (Source: {res.metadata.get('source')}) ---")
        print(res.page_content[:300] + "...")

test_search("What are the guidelines for exercising with hypertension?")

In [10]:
from langgraph.graph import StateGraph, START, END
from state import GraphState
from graphs.nodes.query_analysis import query_analyzer_node
#from graphs.nodes.retriever import retriever_node
from graphs.nodes.generator import generator_node
#from graphs.nodes.web_search import web_search_node
#from graphs.nodes.grade_docs import grader_node




ImportError: cannot import name 'generator_node' from 'graphs.nodes.generator' (c:\Users\odz-2\Desktop\itü\bitirme\2nd part\exercise-chatbot-project\exercise-chatbot-project\graphs\nodes\generator.py)